# Interactive Eigenvalue and Eigenvector Visualization

This notebook provides interactive visualizations to understand eigenvalues, eigenvectors, and their geometric interpretations.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import seaborn as sns

def plot_eigenvectors(matrix, vectors=None, num_points=1000):
    """Plot eigenvectors and their transformations"""
    # Compute eigenvalues and eigenvectors if not provided
    if vectors is None:
        eigenvals, eigenvecs = np.linalg.eig(matrix)
    else:
        eigenvals = np.array([np.linalg.norm(matrix @ v) / np.linalg.norm(v) for v in vectors])
        eigenvecs = np.array(vectors).T
    
    # Create figure
    plt.figure(figsize=(12, 5))
    
    # Plot original vectors and their transformations
    ax1 = plt.subplot(121)
    ax2 = plt.subplot(122)
    
    # Create points along eigenvectors
    t = np.linspace(-2, 2, num_points)
    
    for i in range(len(eigenvals)):
        # Original eigenvector
        v = eigenvecs[:, i]
        points = np.outer(t, v)
        ax1.plot(points[:, 0], points[:, 1], label=f'λ₁ = {eigenvals[i]:.2f}')
        
        # Transformed vector
        transformed = points @ matrix.T
        ax2.plot(transformed[:, 0], transformed[:, 1], label=f'Transformed')
    
    # Plot settings
    for ax in [ax1, ax2]:
        ax.grid(True)
        ax.set_xlim(-3, 3)
        ax.set_ylim(-3, 3)
        ax.set_aspect('equal')
        ax.legend()
    
    ax1.set_title('Original Eigenvectors')
    ax2.set_title('Transformed Vectors')
    
    plt.tight_layout()
    return plt

# Interactive widget for exploring eigenvectors
def eigen_visualization_widget():
    # Create widgets for matrix elements
    a11 = widgets.FloatSlider(value=2.0, min=-2.0, max=2.0, step=0.1, description='a₁₁')
    a12 = widgets.FloatSlider(value=1.0, min=-2.0, max=2.0, step=0.1, description='a₁₂')
    a21 = widgets.FloatSlider(value=1.0, min=-2.0, max=2.0, step=0.1, description='a₂₁')
    a22 = widgets.FloatSlider(value=2.0, min=-2.0, max=2.0, step=0.1, description='a₂₂')
    
    # Create widget for matrix type
    matrix_type = widgets.Dropdown(
        options=['Custom', 'Symmetric', 'Rotation', 'Scaling'],
        value='Custom',
        description='Matrix type:'
    )
    
    # Function to update matrix based on type
    def update_matrix(change):
        if change['new'] == 'Symmetric':
            a11.value = 2.0
            a12.value = 1.0
            a21.value = 1.0
            a22.value = 2.0
        elif change['new'] == 'Rotation':
            angle = np.pi/4
            a11.value = np.cos(angle)
            a12.value = -np.sin(angle)
            a21.value = np.sin(angle)
            a22.value = np.cos(angle)
        elif change['new'] == 'Scaling':
            a11.value = 2.0
            a12.value = 0.0
            a21.value = 0.0
            a22.value = 0.5
    
    matrix_type.observe(update_matrix, 'value')
    
    # Function to update plot
    def update_plot(a11, a12, a21, a22):
        matrix = np.array([[a11, a12],
                          [a21, a22]])
        
        # Compute eigenvalues and eigenvectors
        try:
            eigenvals, eigenvecs = np.linalg.eig(matrix)
            plot_eigenvectors(matrix)
            
            # Print eigenvalue information
            print("\nEigenvalues:")
            for i, (val, vec) in enumerate(zip(eigenvals, eigenvecs.T)):
                print(f"λ{i+1} = {val:.2f}")
                print(f"v{i+1} = [{vec[0]:.2f}, {vec[1]:.2f}]")
                
        except np.linalg.LinAlgError:
            print("Could not compute eigenvalues/eigenvectors")
            
        plt.show()
    
    # Create interactive widget
    interactive_plot = widgets.interactive(
        update_plot,
        a11=a11,
        a12=a12,
        a21=a21,
        a22=a22
    )
    
    # Display widgets
    display(widgets.VBox([matrix_type, interactive_plot]))

# Display the interactive widget
eigen_visualization_widget()

## Special Cases and Properties

Let's explore some special cases and properties of eigenvalues and eigenvectors.

In [ ]:
def explore_special_matrices():
    """Explore eigenvalues and eigenvectors of special matrices"""
    # Create different types of matrices
    matrices = {
        'Identity': np.eye(2),
        'Rotation (90°)': np.array([[0, -1],
                                  [1, 0]]),
        'Scaling': np.array([[2, 0],
                            [0, 0.5]]),
        'Projection': np.array([[1, 0],
                              [0, 0]]),
        'Symmetric': np.array([[2, 1],
                             [1, 2]]),
        'Upper Triangular': np.array([[2, 1],
                                    [0, 3]])
    }
    
    # Plot eigenvectors for each matrix
    fig = plt.figure(figsize=(15, 10))
    
    for i, (name, matrix) in enumerate(matrices.items(), 1):
        # Compute eigenvalues and eigenvectors
        eigenvals, eigenvecs = np.linalg.eig(matrix)
        
        # Create subplot
        ax = fig.add_subplot(2, 3, i)
        
        # Plot eigenvectors
        for j in range(2):
            v = eigenvecs[:, j]
            t = np.linspace(-2, 2, 100)
            points = np.outer(t, v)
            
            # Original vector
            ax.plot(points[:, 0], points[:, 1], 
                   label=f'λ₁ = {eigenvals[j]:.2f}')
            
            # Transformed vector
            transformed = points @ matrix.T
            ax.plot(transformed[:, 0], transformed[:, 1], '--')
        
        ax.grid(True)
        ax.set_xlim(-3, 3)
        ax.set_ylim(-3, 3)
        ax.set_aspect('equal')
        ax.set_title(name)
        ax.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Print eigenvalue properties
    print("\nEigenvalue Properties:")
    for name, matrix in matrices.items():
        eigenvals = np.linalg.eigvals(matrix)
        print(f"\n{name}:")
        print(f"Eigenvalues: {eigenvals}")
        print(f"Trace = {np.trace(matrix):.2f}, Sum of eigenvalues = {np.sum(eigenvals):.2f}")
        print(f"Determinant = {np.linalg.det(matrix):.2f}, Product of eigenvalues = {np.prod(eigenvals):.2f}")

# Run the exploration
explore_special_matrices()

## Applications and Exercises

1. Find matrices with specific eigenvalues
2. Explore the relationship between eigenvalues and matrix properties
3. Investigate repeated eigenvalues
4. Study the effect of matrix operations on eigenvalues

In [ ]:
# Your solutions here
